In [1]:
# Cell 0: Load artifacts from preprocessing notebook

import json, joblib
import numpy as np
import pandas as pd
from pathlib import Path

# dirs
OUT_DIR = Path(r"C:\Users\Nicee\Desktop\kenkyu\gnamboost_dnbr_outputs")
INT_DIR = OUT_DIR / "interim"
FIG_DIR = OUT_DIR / "figs"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# raw X / y
Xtr = joblib.load(INT_DIR / "Xtr.pkl")
Xva = joblib.load(INT_DIR / "Xva.pkl")
Xte = joblib.load(INT_DIR / "Xte.pkl")
ytr = joblib.load(INT_DIR / "ytr.npy")
yva = joblib.load(INT_DIR / "yva.npy")
yte = joblib.load(INT_DIR / "yte.npy")

# transformed X
Xtr_t = joblib.load(INT_DIR / "Xtr_t.npy")
Xva_t = joblib.load(INT_DIR / "Xva_t.npy")
Xte_t = joblib.load(INT_DIR / "Xte_t.npy")

# preprocessing and meta
preproc = joblib.load(INT_DIR / "preproc.joblib")
meta = json.load(open(INT_DIR / "meta.json", "r", encoding="utf-8"))

feature_cols = meta["feature_cols"]
num_cols = meta["num_cols"]
bin_cols = meta["bin_cols"]
cat_cols = meta["cat_cols"]

df = pd.read_parquet(INT_DIR / "df_filtered.parquet")

print("Loaded shapes:")
print("Xtr:", Xtr.shape, "Xva:", Xva.shape, "Xte:", Xte.shape)
print("Xtr_t:", Xtr_t.shape, "Xva_t:", Xva_t.shape, "Xte_t:", Xte_t.shape)
print("y:", ytr.shape, yva.shape, yte.shape)


Loaded shapes:
Xtr: (1526106, 31) Xva: (381469, 31) Xte: (476863, 31)
Xtr_t: (1526106, 177) Xva_t: (381469, 177) Xte_t: (476863, 177)
y: (1526106,) (381469,) (476863,)


In [2]:
# Cell 9: Logistic & XGBoost baselines

import numpy as np, joblib
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
import xgboost as xgb

# logistic
basic_cols = ["IDADEMAE", "SEMAGESTAC", "PESO"]
Xtr_basic = Xtr[basic_cols].copy()
Xva_basic = Xva[basic_cols].copy()
Xte_basic = Xte[basic_cols].copy()

pipe_logit = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
])

Xtr_logit = pipe_logit.fit_transform(Xtr_basic)
Xva_logit = pipe_logit.transform(Xva_basic)
Xte_logit = pipe_logit.transform(Xte_basic)

logit = LogisticRegression(
    penalty="l2",
    C=0.2, 
    solver="lbfgs",
    max_iter=1000,
)

logit.fit(Xtr_logit, ytr)
p_va_logit = logit.predict_proba(Xva_logit)[:, 1]
p_te_logit_raw = logit.predict_proba(Xte_logit)[:, 1]

iso_logit = IsotonicRegression(out_of_bounds="clip").fit(p_va_logit, yva)
p_logit = iso_logit.transform(p_te_logit_raw)

print("Logistic (3 vars): AUROC=",
      roc_auc_score(yte, p_logit),
      "AP=",
      average_precision_score(yte, p_logit))

# XGBoost
pos_rate_all = float(ytr.mean())
scale_pos = (1.0 - pos_rate_all) / max(pos_rate_all, 1e-6)

xgb_model = xgb.XGBClassifier(
    grow_policy="lossguide",
    max_depth=0,
    max_leaves=80,
    learning_rate=0.02,
    n_estimators=12000,
    subsample=0.8,
    colsample_bytree=0.8,
    colsample_bylevel=0.8,
    min_child_weight=10,
    reg_alpha=0.1,
    reg_lambda=2.0,
    gamma=0.0,
    max_bin=256,
    max_delta_step=1,
    tree_method="hist",
    n_jobs=-1,
    random_state=meta["seed"],
    early_stopping_rounds=1000,
    eval_metric="auc",
    scale_pos_weight=scale_pos,
)

xgb_model.fit(Xtr_t, ytr, eval_set=[(Xva_t, yva)], verbose=False)
best_iter = xgb_model.best_iteration
booster = xgb_model.get_booster()

dva = xgb.DMatrix(Xva_t)
dte = xgb.DMatrix(Xte_t)

p_va_xgb = booster.predict(dva, iteration_range=(0, best_iter + 1))
p_te_xgb_raw = booster.predict(dte, iteration_range=(0, best_iter + 1))

iso_xgb = IsotonicRegression(out_of_bounds="clip").fit(p_va_xgb, yva)
p_xgb = iso_xgb.transform(p_te_xgb_raw)

print("XGB: AUROC=",
      roc_auc_score(yte, p_xgb),
      "AP=",
      average_precision_score(yte, p_xgb))

# save
joblib.dump(p_logit, INT_DIR / "p_logit.npy")
joblib.dump(p_xgb,   INT_DIR / "p_xgb.npy")
joblib.dump(logit,   INT_DIR / "logit_model.joblib")
booster.save_model(str(INT_DIR / "xgb_booster.json"))
joblib.dump({"best_iteration": best_iter}, INT_DIR / "xgb_info.joblib")


Logistic (3 vars): AUROC= 0.6216665557825451 AP= 0.023254804673634256
XGB: AUROC= 0.7290596035392394 AP= 0.0835858929342947


['C:\\Users\\Nicee\\Desktop\\kenkyu\\gnamboost_dnbr_outputs\\interim\\xgb_info.joblib']

In [3]:
# Cell 10: GNAM training

import os, contextlib, time
import numpy as np, joblib, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.isotonic import IsotonicRegression

os.environ["OMP_NUM_THREADS"] = "12"
os.environ["MKL_NUM_THREADS"] = "12"
try:
    torch.set_num_threads(12)
    torch.set_num_interop_threads(4)
except Exception:
    pass

def get_amp_ctx():
    try:
        return torch.autocast(device_type="cpu", dtype=torch.bfloat16)
    except Exception:
        return contextlib.nullcontext()

amp_ctx = get_amp_ctx()

class ExULayer(nn.Module):
    def __init__(self, nonlin="softplus"):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(1))
        self.b = nn.Parameter(torch.zeros(1))
        self.nonlin = nonlin
    def forward(self, x):
        z = torch.exp(self.w) * (x - self.b)
        if self.nonlin == "softplus":
            return F.softplus(z)
        if self.nonlin == "tanh":
            return torch.tanh(z)
        return F.relu(z)

class FeatureNet(nn.Module):
    def __init__(self, hidden=96, dropout=0.05):
        super().__init__()
        self.exu = ExULayer("softplus")
        self.fc1 = nn.Linear(1, hidden)
        self.fc2 = nn.Linear(hidden, hidden // 2)
        self.fc3 = nn.Linear(hidden // 2, 1)
        self.dp = nn.Dropout(dropout)
    def forward(self, x):
        h = self.exu(x)
        h = F.relu(self.fc1(h))
        h = self.dp(F.relu(self.fc2(h)))
        return self.fc3(h)

class GNAM(nn.Module):
    def __init__(self, n_features, hidden=96, dropout=0.05):
        super().__init__()
        self.bias = nn.Parameter(torch.zeros(1))
        self.fnets = nn.ModuleList([FeatureNet(hidden, dropout) for _ in range(n_features)])
    def forward(self, x):
        outs = [self.fnets[j](x[:, j:j+1]) for j in range(x.shape[1])]
        eta = self.bias + torch.stack(outs, dim=2).sum(dim=2)
        return eta.squeeze(1)

def to_loader(X, y, batch=4096, shuffle=True):
    ds = TensorDataset(torch.tensor(X, dtype=torch.float32),
                       torch.tensor(y, dtype=torch.float32))
    return DataLoader(ds, batch_size=batch, shuffle=shuffle,
                      drop_last=False, num_workers=0)

def train_gnam(model, Xtr_mat, ytr_vec, Xva_mat, yva_vec,
               max_epochs=180, lr=7e-4, weight_decay=1e-6,
               patience=40, device="cpu", pos_weight=None,
               logit_clamp=None, grad_clip_norm=3.0, report_every=10):

    if pos_weight is not None:
        criterion = nn.BCEWithLogitsLoss(
            pos_weight=torch.tensor(float(pos_weight), device=device)
        )
    else:
        criterion = nn.BCEWithLogitsLoss()

    tr_loader = to_loader(Xtr_mat, ytr_vec, batch=4096, shuffle=True)
    va_loader = to_loader(Xva_mat, yva_vec, batch=4096, shuffle=False)

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_metric = -np.inf
    best_state = None
    bad_epochs = 0

    model.to(device)

    for ep in range(max_epochs):
        t0 = time.time()
        model.train()
        tr_loss_sum, tr_n = 0.0, 0

        for xb, yb in tr_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with amp_ctx:
                logits = model(xb)
                if logit_clamp is not None:
                    logits = torch.clamp(logits, -abs(logit_clamp), abs(logit_clamp))
                loss = criterion(logits, yb)
            loss.backward()
            if grad_clip_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
            opt.step()
            tr_loss_sum += float(loss.item()) * len(xb)
            tr_n += len(xb)

        tr_loss = tr_loss_sum / max(tr_n, 1)

        # validation
        model.eval()
        va_loss_sum, va_n = 0.0, 0
        logits_all, y_all = [], []
        with torch.no_grad():
            for xb, yb in va_loader:
                xb, yb = xb.to(device), yb.to(device)
                with amp_ctx:
                    logits = model(xb)
                    if logit_clamp is not None:
                        logits = torch.clamp(logits, -abs(logit_clamp), abs(logit_clamp))
                vloss = F.binary_cross_entropy_with_logits(logits, yb).item()
                va_loss_sum += vloss * len(xb)
                va_n += len(xb)
                logits_all.append(logits.detach().cpu().numpy().ravel())
                y_all.append(yb.detach().cpu().numpy().ravel())

        avg_val_loss = va_loss_sum / max(va_n, 1)
        y_cat = np.concatenate(y_all) if y_all else np.array([])
        p_cat = 1.0 / (1.0 + np.exp(-np.concatenate(logits_all))) if logits_all else np.array([])

        if y_cat.size > 0 and np.unique(y_cat).size > 1:
            auroc = roc_auc_score(y_cat, p_cat)
            ap = average_precision_score(y_cat, p_cat)
        else:
            auroc, ap = np.nan, np.nan

        metric = auroc if np.isfinite(auroc) else -np.inf

        if ((ep + 1) % report_every) == 0:
            print(
                f"Epoch {ep+1:03d}/{max_epochs} | "
                f"train_loss={tr_loss:.5f} | val_loss={avg_val_loss:.5f} | "
                f"AUROC={auroc:.4f} | AP={ap:.4f} | "
                f"best_AUROC={best_metric:.4f} | bad_epochs={bad_epochs} | "
                f"time={time.time()-t0:.1f}s"
            )

        # early stop
        if metric > best_metric + 1e-4:
            best_metric = metric
            bad_epochs = 0
            best_state = {k: v.detach().cpu().clone()
                          for k, v in model.state_dict().items()}
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print(f"Early stop @ epoch {ep+1}, best_AUROC={best_metric:.5f}")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

SUB_N = 400_000
if Xtr_t.shape[0] > SUB_N:
    rng = np.random.RandomState(meta["seed"])
    idx_sub = rng.choice(Xtr_t.shape[0], size=SUB_N, replace=False)
    Xtr_t_sub = Xtr_t[idx_sub]
    ytr_sub = ytr[idx_sub]
else:
    Xtr_t_sub, ytr_sub = Xtr_t, ytr

device = "cuda" if torch.cuda.is_available() else "cpu"

gnam = GNAM(n_features=Xtr_t.shape[1], hidden=96, dropout=0.05)
gnam = train_gnam(
    gnam,
    Xtr_t_sub, ytr_sub,
    Xva_t, yva,
    max_epochs=180,
    lr=7e-4,
    weight_decay=1e-6,
    patience=40,
    device=device,
    pos_weight=None,
    logit_clamp=None,
    grad_clip_norm=3.0,
    report_every=10,
)

torch.save(gnam.state_dict(), INT_DIR / "gnam_best.pt")

# predictions
import scipy.special as sps

gnam.eval()
with torch.no_grad(), amp_ctx:
    eta_tr = gnam(torch.tensor(Xtr_t, dtype=torch.float32, device=device)).cpu().numpy().astype(np.float32)
    eta_va = gnam(torch.tensor(Xva_t, dtype=torch.float32, device=device)).cpu().numpy().astype(np.float32)
    eta_te = gnam(torch.tensor(Xte_t, dtype=torch.float32, device=device)).cpu().numpy().astype(np.float32)

p_tr_gnam = sps.expit(eta_tr)
p_va_gnam = sps.expit(eta_va)
p_te_gnam = sps.expit(eta_te)

iso_gnam = IsotonicRegression(out_of_bounds="clip").fit(p_va_gnam, yva)
p_gnam = iso_gnam.transform(p_te_gnam)

print("GNAM valid (raw): AUROC=",
      roc_auc_score(yva, p_va_gnam),
      "AP=",
      average_precision_score(yva, p_va_gnam))
print("GNAM test (calib): AUROC=",
      roc_auc_score(yte, p_gnam),
      "AP=",
      average_precision_score(yte, p_gnam))

joblib.dump({"eta_tr": eta_tr, "eta_va": eta_va, "eta_te": eta_te}, INT_DIR / "gnam_eta.joblib")
joblib.dump(p_gnam, INT_DIR / "p_gnam.npy")



Epoch 010/180 | train_loss=0.05802 | val_loss=0.05810 | AUROC=0.7098 | AP=0.0658 | best_AUROC=0.7101 | bad_epochs=2 | time=78.0s
Epoch 020/180 | train_loss=0.05735 | val_loss=0.05793 | AUROC=0.7099 | AP=0.0673 | best_AUROC=0.7113 | bad_epochs=0 | time=78.3s
Epoch 030/180 | train_loss=0.05654 | val_loss=0.05801 | AUROC=0.7093 | AP=0.0673 | best_AUROC=0.7113 | bad_epochs=10 | time=77.9s
Epoch 040/180 | train_loss=0.05713 | val_loss=0.05785 | AUROC=0.7099 | AP=0.0674 | best_AUROC=0.7113 | bad_epochs=20 | time=107.5s
Epoch 050/180 | train_loss=0.05657 | val_loss=0.05791 | AUROC=0.7105 | AP=0.0681 | best_AUROC=0.7113 | bad_epochs=30 | time=107.3s
Early stop @ epoch 59, best_AUROC=0.71128
GNAM valid (raw): AUROC= 0.711283237056559 AP= 0.06721903696098361
GNAM test (calib): AUROC= 0.7148797820758007 AP= 0.0708614318580256


['C:\\Users\\Nicee\\Desktop\\kenkyu\\gnamboost_dnbr_outputs\\interim\\p_gnam.npy']

In [4]:
# Cell 11: GNAM–Boost

import xgboost as xgb
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, log_loss
import numpy as np, joblib

# load GNAM logits
etas = joblib.load(INT_DIR / "gnam_eta.joblib")
eta_tr, eta_va, eta_te = etas["eta_tr"], etas["eta_va"], etas["eta_te"]

# DMatrices with base_margin from GNAM
dtr2 = xgb.DMatrix(Xtr_t, label=ytr)
dtr2.set_base_margin(eta_tr.astype(np.float32))

dva2 = xgb.DMatrix(Xva_t, label=yva)
dva2.set_base_margin(eta_va.astype(np.float32))

dte2 = xgb.DMatrix(Xte_t, label=yte)
dte2.set_base_margin(eta_te.astype(np.float32))

pos_rate = float(ytr.mean())
print("Train size:", dtr2.num_row(), "| pos_rate:", pos_rate)

params2 = {
    "objective": "binary:logistic",
    "eval_metric": ["aucpr", "auc", "logloss"],
    "grow_policy": "lossguide",
    "max_depth": 0,
    "max_leaves": 128,
    "eta": 0.03,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "colsample_bylevel": 0.9,
    "min_child_weight": 1,
    "reg_alpha": 0.0,
    "reg_lambda": 1.0,
    "max_delta_step": 0,
    "scale_pos_weight": 1.0,
    "tree_method": "hist"
}

bst2 = xgb.train(
    params2,
    dtr2,
    num_boost_round=8000,
    evals=[(dtr2, "train"), (dva2, "valid")],
    early_stopping_rounds=1000,
    verbose_eval=200
)

best_iter = bst2.best_iteration if hasattr(bst2, "best_iteration") else None
print("Best iteration:", best_iter)

# raw predictions
p_va_gb_raw = bst2.predict(
    dva2,
    iteration_range=(0, best_iter + 1) if best_iter is not None else None
)
p_te_gb_raw = bst2.predict(
    dte2,
    iteration_range=(0, best_iter + 1) if best_iter is not None else None
)

print("GNAM–Boost (valid raw): AUROC = {:.4f}  AP = {:.4f}".format(
    roc_auc_score(yva, p_va_gb_raw),
    average_precision_score(yva, p_va_gb_raw)
))

# isotonic calibration on valid
iso_gb = IsotonicRegression(out_of_bounds="clip").fit(p_va_gb_raw, yva)
p_final = iso_gb.transform(p_te_gb_raw)

print(
    "GNAM–Boost (test calib): AUROC = {:.4f}  AP = {:.4f}  Brier = {:.4f}  LogLoss = {:.4f}"
    .format(
        roc_auc_score(yte, p_final),
        average_precision_score(yte, p_final),
        brier_score_loss(yte, p_final),
        log_loss(yte, p_final)
    )
)

# save artifacts
bst2.save_model(str(INT_DIR / "gnamboost_bst.json"))
joblib.dump(p_final, INT_DIR / "p_final.npy")


Train size: 1526106 | pos_rate: 0.011275101467394794
[0]	train-aucpr:0.07696	train-auc:0.71951	train-logloss:0.05665	valid-aucpr:0.06834	valid-auc:0.71193	valid-logloss:0.05770
[200]	train-aucpr:0.14195	train-auc:0.78817	train-logloss:0.05293	valid-aucpr:0.08850	valid-auc:0.72332	valid-logloss:0.05679
[400]	train-aucpr:0.18402	train-auc:0.83373	train-logloss:0.05054	valid-aucpr:0.08927	valid-auc:0.72381	valid-logloss:0.05676
[600]	train-aucpr:0.23026	train-auc:0.86619	train-logloss:0.04841	valid-aucpr:0.08902	valid-auc:0.72213	valid-logloss:0.05682
[800]	train-aucpr:0.27837	train-auc:0.88997	train-logloss:0.04649	valid-aucpr:0.08866	valid-auc:0.72122	valid-logloss:0.05687
[1000]	train-aucpr:0.32760	train-auc:0.90849	train-logloss:0.04472	valid-aucpr:0.08846	valid-auc:0.71956	valid-logloss:0.05693
[1200]	train-aucpr:0.37749	train-auc:0.92332	train-logloss:0.04306	valid-aucpr:0.08804	valid-auc:0.71804	valid-logloss:0.05701
[1379]	train-aucpr:0.42220	train-auc:0.93507	train-logloss:0.0416

['C:\\Users\\Nicee\\Desktop\\kenkyu\\gnamboost_dnbr_outputs\\interim\\p_final.npy']